In [35]:
import pandas as pd
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import BasicTickFormatter, Label, Title, NumeralTickFormatter
# HoverTool allows you to set a tooltips property which takes a list of tuples
from bokeh.models.tools import HoverTool
# a pre-made five color pallette, one of Bokeh’s many pre-made color palettes
from bokeh.palettes import Spectral11
# a helper method for mapping colors to bars in a bar-charts
from bokeh.transform import factor_cmap
import math

datapath = 'datapandas/'
inputFile = 'ABS2021NeedForAssistance.xlsx'     # need for assistance

# Function "output_file" defines the output name and how the visualization will be rendered by bokeh, eg html for this one.
# File will be saved in the main project folder
output_file('codeflask/templates/assistanceByNeedForAssistance.html')

# Read the two input excel files
dfNeedForAssistance = pd.read_excel(datapath+inputFile)
#print(dfPopAgeByGrp,'\n')

# Group population data by AgeGroup
dfMerged = dfNeedForAssistance.groupby('AgeGroup')[['NeedAssistance']].sum().reset_index()
#print(dfMerged,'\n')

# Re-order data, move 6 2nd row
df1 = dfMerged.drop([0,6])
df2 = dfMerged.iloc[[0,6], :]
dfGrouped=pd.concat([df2,df1], ignore_index = True)  # Concatenate dataframes
#print(dfGrouped,'\n')

#-----------------------------------------------------------------------------------------------------------

# Set dataframe into 'column data source' so it can be processed by bokeh
source = ColumnDataSource(dfGrouped)
# Get list of AgeGroup values for horizontal bar chart (x-axis)
AgeGrp = source.data['AgeGroup'].tolist()
#print(AgeGrp,'\n')

#-----------------------------------------------------------------------------------------------------------

p = figure(y_range=AgeGrp, width=700, height=400
           , background_fill_color="#292929" #404040
           , border_fill_color = "black"
           , outline_line_color="white"
           , outline_line_width = 0
           #, outline_line_alpha = 0.3
           )

# Plot data as individually colored bars and add basic labels.
# To color the bars we use the factor_cmap helper function.
# This creates a special color map that matches an individual color to each Category (ageDesc)
color_map = factor_cmap(field_name='AgeGroup', # create a special color map that matches an individual color to each category
                        palette=Spectral11,
                        factors=AgeGrp)      # factors in bokeh refers to Categories.

# Call vbar method to create a vertical bar Glyph
p.hbar(y='AgeGroup', right='NeedAssistance', source=source, height=0.6, color=color_map)
# Add labels
p.title.text ='Core Activity Need for Assistance by Age Group as at 2021 Census'
p.xaxis.axis_label = 'People Who Need Assistance'
p.yaxis.axis_label = 'Age Group'

#------------------------------------------------------------------------
# Specify Axis labels
##axis_label_text_color = 'red'
##axis_line_color = 'red'
p.yaxis.major_label_text_color = "#e0e0e0"        #ok
p.yaxis.axis_label_text_color = "#e0e0e0"         #ok
p.xaxis.major_label_text_color = "#e0e0e0"        #ok
p.yaxis.major_label_text_font_size = "9pt"
p.xaxis.major_label_text_font_size = "8pt"
p.xaxis.axis_label_text_color = "#e0e0e0"         #ok

# Change orientation of x-axis labels
#p.xaxis.major_label_orientation = math.pi/3
#p.xaxis.major_label_orientation = "vertical"

# Change scientific notation in y-axis
p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")

# Chart title
p.title.text_color = "#e0e0e0"
#Hide Gridlines
p.xgrid.grid_line_color = "#e0e0e0"
p.ygrid.grid_line_color = "#e0e0e0"
#--------------------------------------------------------------------------
# Add hover
hover = HoverTool()    # HoverTool allows you to set a tooltips property which takes a list of tuples
hover.tooltips = [("","Age Group:@AgeGroup;  Need Assistance:@NeedAssistance{0,0}")]
#hover.mode = 'vline'   # tell the popup to show when a vertical line crosses a glyph
p.add_tools(hover)     # add hover tool in the toolbar.

#show(p)
save(p)

#end of file